### Task 1 Data cleaning

In [1]:
import pandas as pd
import math
import numpy as np

In [2]:
data1 = pd.read_excel("/Users/hyosangkim/Downloads/resources/task_1/supplier_data1.xlsx")

In [3]:
data2 = pd.read_excel("/Users/hyosangkim/Downloads/resources/task_1/supplier_data2.xlsx")

Cleaning
* check the unique values for the columns, check the inconsistency
* check whether the comma(,) and period(.) is mix used
* unify the language to english using the mapping (no automation because of technical terminologies, unifying translating value), with lower case and stripping
* standardize unit of numerical values

Integration
* data2 Material can be integrated with the data1 Grade
* data2 Description can be integrated with the data1 description
* data2 Articla ID should be the index when integrating data1 and data2

##### cleaning 1: unify text

In [4]:
# Check the columns to unify the text
data1['Finish'].value_counts()

Finish
gebeizt und geglüht    19
gebeizt                17
ungebeizt              14
Name: count, dtype: int64

In [5]:
# Check the columns to unify the text
data1['Description'].value_counts()

Description
Längs- oder Querisse                  23
Sollmasse (Gewicht) unterschritten    16
Kantenfehler - FS-Kantenrisse         11
Name: count, dtype: int64

In [6]:
# Check the columns to unify the text
data2['Description'].value_counts()

Description
Material is not Oiled    18
Material is Oiled        17
Material is Painted      15
Name: count, dtype: int64

In [7]:
# Mapping process
mapping = {
    "gebeizt und geglüht": "pickled and annealed",
    "gebeizt": "pickled",
    "ungebeizt": "unpickled",
    "Material is not Oiled": "not oiled",
    "Material is Oiled": "oiled", 
    "Material is Painted": "painted",
    "Längs- oder Querisse": "longitudinal or transverse cracks",
    "Kantenfehler - FS-Kantenrisse": "edge defect - edge cracks",
    "Sollmasse (Gewicht) unterschritten": "target weight unreached",
}

def unify_text(value):
    if pd.isna(value):
        return "NULL VALUE FOUND"
    return mapping.get(value.strip(), "EXCEPTIONS")

data1['Finish'] = data1['Finish'].apply(unify_text)
data2['Description'] = data2['Description'].apply(unify_text)
data1['Description'] = data1['Description'].apply(unify_text)

##### cleaning 2: unify the numerical unit
1. data1: 
    - thickness and width to float
    - quantity to int
2. data2: 
    - no numerical unit to clean

In [8]:
data1.head()

,Quality/Choice,Grade,Finish,Thickness (mm),Width (mm),Description,Gross weight (kg),RP02,RM,Quantity,AG,AI
0,3rd,C200S,pickled and annealed,2.77,1100,longitudinal or transverse cracks,13983,333.6,606.2,0.00,16.11,0.0054
1,3rd,C300S,unpickled,2.65,1075,longitudinal or transverse cracks,13047,717.7,0.0,0.00,16.11,0.0046
2,3rd,C100S,pickled and annealed,2.20,1100,edge defect - edge cracks,14155,368.9,0.0,10.84,0.00,0.0061
3,2nd,C100S,pickled,2.86,1100,longitudinal or transverse cracks,11381,368.9,601.7,22.87,0.00,0.0062
4,1st,C300S,unpickled,2.88,1050,target weight unreached,10072,0.0,1213.0,22.87,0.00,0.0041


In [9]:
# clean numerical values
def normalize_quantity(qty):
    if pd.isna(qty):
        return "NULL VALUE FOUND"
    return int(math.floor(float(qty)))

data1['Width (mm)'] = data1['Width (mm)'].astype(float)
data1['Quantity'] = data1['Quantity'].apply(normalize_quantity)

##### Cleaning 3: Standardize the column name
1. data1: 
     - Gross weight (kg) to Weight(kg) to match with data2
     

In [10]:
data1.rename(columns={'Gross weight (kg)': 'Weight (kg)'}, inplace=True)

##### Cleaning 4: check the exact duplicates

In [11]:
data1.duplicated().value_counts()

False    50
Name: count, dtype: int64

In [12]:
data2.duplicated().value_counts()

False    50
Name: count, dtype: int64

##### Join
- Find key to join: data1 grade and data2 material can be the same column

In [13]:
data1['Grade'].value_counts()

Grade
C100S    26
C300S    14
C200S    10
Name: count, dtype: int64

In [14]:
data2['Material'].value_counts()

Material
S235JR          19
HDC             13
DX51D +AZ150    10
DX51D +Z140      8
Name: count, dtype: int64

In [15]:
data1.rename(columns={'Grade': 'Grade/Material'}, inplace=True)
data2.rename(columns={'Material': 'Grade/Material'}, inplace=True)

In [16]:
data1.head()

,Quality/Choice,Grade/Material,Finish,Thickness (mm),Width (mm),Description,Weight (kg),RP02,RM,Quantity,AG,AI
0,3rd,C200S,pickled and annealed,2.77,1100.0,longitudinal or transverse cracks,13983,333.6,606.2,0,16.11,0.0054
1,3rd,C300S,unpickled,2.65,1075.0,longitudinal or transverse cracks,13047,717.7,0.0,0,16.11,0.0046
2,3rd,C100S,pickled and annealed,2.20,1100.0,edge defect - edge cracks,14155,368.9,0.0,10,0.00,0.0061
3,2nd,C100S,pickled,2.86,1100.0,longitudinal or transverse cracks,11381,368.9,601.7,22,0.00,0.0062
4,1st,C300S,unpickled,2.88,1050.0,target weight unreached,10072,0.0,1213.0,22,0.00,0.0041


In [17]:
data2.head()

,Grade/Material,Description,Article ID,Weight (kg),Quantity,Reserved
0,HDC,oiled,23048203,24469,52,NOT RESERVED
1,S235JR,oiled,23040547,16984,41,NOT RESERVED
2,S235JR,painted,23046057,9162,28,NOT RESERVED
3,DX51D +AZ150,oiled,23041966,12119,66,VANILLA
4,HDC,painted,23043884,17260,26,NOT RESERVED


In [18]:
final_cols = [
    "Quality/Choice",
    "Grade/Material", "Finish","Description",
    "Thickness (mm)","Width (mm)",
    "Weight (kg)","Quantity","Reserved",
    "Article ID", "RP02","RM","AG","AI"
]

# ensure all columns exist in both frames
for c in final_cols:
    if c not in data1.columns: data1[c] = np.nan
    if c not in data2.columns: data2[c] = np.nan

data1 = data1[final_cols]
data2 = data2[final_cols]

inventory = pd.concat([data1, data2], ignore_index=True)
inventory

,Quality/Choice,Grade/Material,Finish,Description,Thickness (mm),Width (mm),Weight (kg),Quantity,Reserved,Article ID,RP02,RM,AG,AI
0,3rd,C200S,pickled and annealed,longitudinal or transverse cracks,2.77,1100.0,13983,0,NaN,NaN,333.6,606.2,16.11,0.0054
1,3rd,C300S,unpickled,longitudinal or transverse cracks,2.65,1075.0,13047,0,NaN,NaN,717.7,0.0,16.11,0.0046
2,3rd,C100S,pickled and annealed,edge defect - edge cracks,2.20,1100.0,14155,10,NaN,NaN,368.9,0.0,0.00,0.0061
3,2nd,C100S,pickled,longitudinal or transverse cracks,2.86,1100.0,11381,22,NaN,NaN,368.9,601.7,0.00,0.0062
4,1st,C300S,unpickled,target weight unreached,2.88,1050.0,10072,22,NaN,NaN,0.0,1213.0,0.00,0.0041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,NaN,DX51D +Z140,NaN,not oiled,NaN,NaN,9940,96,NOT RESERVED,23047543.0,NaN,NaN,NaN,NaN
96,NaN,DX51D +AZ150,NaN,painted,NaN,NaN,23884,51,NOT RESERVED,23045292.0,NaN,NaN,NaN,NaN
97,NaN,S235JR,NaN,oiled,NaN,NaN,5302,12,VANILLA,23047408.0,NaN,NaN,NaN,NaN
98,NaN,DX51D +Z140,NaN,oiled,NaN,NaN,16927,69,VANILLA,23043348.0,NaN,NaN,NaN,NaN


In [19]:
inventory.rename(columns={
    "Quality/Choice": "quality/choice",
    "Grade/Material": "grade/material",
    "Finish": "finish",
    "Description": "description", 
    "Thickness (mm)": "thickness_mm",
    "Width (mm)": "width_mm",
    "Weight (kg)": "weight_kg",
    "Quantity": "quantity",
    "Reserved": "reserved",
    "Article ID": "original_article_id",
}, inplace=True)

In [20]:
inventory

,quality/choice,grade/material,finish,description,thickness_mm,width_mm,weight_kg,quantity,reserved,original_article_id,RP02,RM,AG,AI
0,3rd,C200S,pickled and annealed,longitudinal or transverse cracks,2.77,1100.0,13983,0,NaN,NaN,333.6,606.2,16.11,0.0054
1,3rd,C300S,unpickled,longitudinal or transverse cracks,2.65,1075.0,13047,0,NaN,NaN,717.7,0.0,16.11,0.0046
2,3rd,C100S,pickled and annealed,edge defect - edge cracks,2.20,1100.0,14155,10,NaN,NaN,368.9,0.0,0.00,0.0061
3,2nd,C100S,pickled,longitudinal or transverse cracks,2.86,1100.0,11381,22,NaN,NaN,368.9,601.7,0.00,0.0062
4,1st,C300S,unpickled,target weight unreached,2.88,1050.0,10072,22,NaN,NaN,0.0,1213.0,0.00,0.0041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,NaN,DX51D +Z140,NaN,not oiled,NaN,NaN,9940,96,NOT RESERVED,23047543.0,NaN,NaN,NaN,NaN
96,NaN,DX51D +AZ150,NaN,painted,NaN,NaN,23884,51,NOT RESERVED,23045292.0,NaN,NaN,NaN,NaN
97,NaN,S235JR,NaN,oiled,NaN,NaN,5302,12,VANILLA,23047408.0,NaN,NaN,NaN,NaN
98,NaN,DX51D +Z140,NaN,oiled,NaN,NaN,16927,69,VANILLA,23043348.0,NaN,NaN,NaN,NaN


In [21]:
inventory.to_csv("inventory_dataset.csv", index=False)